# prepare data 

In [43]:
import pandas as pd
from datetime import timedelta

### Пример данных

In [44]:
# Примерные входные данные (замените своими)
# df_investments: date, amount
df_investments = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'amount': [150]*36
})

# df_rates: date, rate (где rate - годовая ставка, например 0.06 для 6%)
df_rates = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'rate': [0.10, 0.10, 0.10] * 12
    #'rate': [0.06, 0.05, 0.07] * 12
})

In [45]:
df_investments

,date,amount
0,2020-01-01,150
1,2020-02-01,150
2,2020-03-01,150
3,2020-04-01,150
4,2020-05-01,150
5,2020-06-01,150
6,2020-07-01,150
7,2020-08-01,150
8,2020-09-01,150
9,2020-10-01,150


### Чтение данных 

In [46]:
# df_investments.to_excel('investments.xlsx')

df_rates = pd.read_excel("Процентная ставка вклады 2010-2024.xlsx")
df_rates = df_rates.rename(columns={'Декада':'date', "Ставка, %":"rate"})

df_investments = pd.read_excel("Вклады.xlsx")


df_rates['rate'] = df_rates['rate'] * 0.01

### Глобальные настройки.

In [47]:
# Параметры
n = 45000  # минимальная сумма для открытия вклада
investment_duration = 12  # продолжительность вклада в месяцах

# Объединяем данные по датам
df = pd.merge(df_investments, df_rates, on='date', how='left')

In [48]:
if df.isna().sum().sum() != 0:
    raise ValueError("Есть пропуск. Проверить данные.")

# Обработка вкладов, первый собранные вклады.

In [49]:
# Рассчитаем доходность по вкладам
def calculate_income(df, min_sum, duration):
    total_income = []
    balance = 0  # текущий баланс для инвестирования
    start_dates = []

    for i, row in df.iterrows():
        balance += row['amount']
        if balance >= min_sum:
            # Открываем вклад
            start_date = row['date']
            end_date = start_date + pd.DateOffset(months=duration)
            deposit_rate = df[(df['date'] == start_date)]['rate'].mean()
            # deposit_rate = df[(df['date'] >= start_date) & (df['date'] < end_date)]['rate'].mean()
            print("deposit_rate", deposit_rate)
            
            if pd.notna(deposit_rate):
                income = balance * deposit_rate * (duration / 12)
                total_income.append({
                    'start_date': start_date,
                    'end_date': end_date,
                    'principal': balance,
                    'rate': deposit_rate,
                    'income': income
                })
            
            # Обнуляем баланс после открытия вклада
            balance = 0
    
    return pd.DataFrame(total_income)

# Вычисление доходности первичных вкладов
df_income = calculate_income(df, n, investment_duration)
print(df_income)


deposit_rate 0.09514
deposit_rate 0.09538
deposit_rate 0.09936
deposit_rate 0.094
deposit_rate 0.0965
deposit_rate 0.0994
deposit_rate 0.0926
deposit_rate 0.08664999999999999
deposit_rate 0.08310000000000001
deposit_rate 0.08615
deposit_rate 0.09005
deposit_rate 0.09502000000000001
deposit_rate 0.15325
deposit_rate 0.13175
deposit_rate 0.11042
deposit_rate 0.10352
deposit_rate 0.09999000000000001
deposit_rate 0.09648999999999999
deposit_rate 0.0913
deposit_rate 0.0866
deposit_rate 0.084
deposit_rate 0.07798000000000001
deposit_rate 0.07363
deposit_rate 0.07237
deposit_rate 0.07245
deposit_rate 0.064
deposit_rate 0.06394
deposit_rate 0.06798
deposit_rate 0.07529
deposit_rate 0.07535
deposit_rate 0.07135
deposit_rate 0.06478
deposit_rate 0.059269999999999996
deposit_rate 0.05427
deposit_rate 0.04629
deposit_rate 0.04326
deposit_rate 0.04486
deposit_rate 0.046340000000000006
deposit_rate 0.053
deposit_rate 0.06433
deposit_rate 0.07738
deposit_rate 0.16579999999999998
deposit_rate 0.07645


In [50]:
max_date = df['date'].max() + pd.DateOffset(months=1)
max_date

Timestamp('2024-12-01 00:00:00')

In [51]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Обработка первичных вкладов.
# Убираю периоды "будущей доходности".
# max_date = df['date'].max()
correct_income = []
for i, row in df_income.iterrows():
    if max_date < row["end_date"]:
        # n_month = relativedelta(max_date, )
        n_month = (max_date.year - row["start_date"].year) * 12 + (max_date.month - row["start_date"].month)
        row["income"] = n_month / 12 * row["income"]
        row["end_date"] = max_date
        
        correct_income.append(row)
    else:
        correct_income.append(row)
        

In [52]:
max_date, row["end_date"]

(Timestamp('2024-12-01 00:00:00'), Timestamp('2024-12-01 00:00:00'))

In [53]:
df_income_correct = pd.DataFrame(correct_income)

In [54]:
df_income_correct

,start_date,end_date,principal,rate,income
0,2012-01-01,2013-01-01,100000,0.09514,9514.0000
1,2012-04-01,2013-04-01,45000,0.09538,4292.1000
2,2012-07-01,2013-07-01,45000,0.09936,4471.2000
3,2012-10-01,2013-10-01,45000,0.09400,4230.0000
4,2013-01-01,2014-01-01,45000,0.09650,4342.5000
5,2013-04-01,2014-04-01,45000,0.09940,4473.0000
6,2013-07-01,2014-07-01,45000,0.09260,4167.0000
7,2013-10-01,2014-10-01,45000,0.08665,3899.2500
8,2014-01-01,2015-01-01,45000,0.08310,3739.5000
9,2014-04-01,2015-04-01,45000,0.08615,3876.7500


# Расчёт повторных инвестиций.

In [70]:
income_new_all = []
income_calculate = []
for i, row in df_income_correct.iterrows():
    #row["end_date"]
    income_all_reinvest = []
    end_date_iter = row['end_date']
    total_money = row["principal"] + row["income"]
    while True:
        if end_date_iter < max_date:
            n_month_for_end = (max_date.year - end_date_iter.year) * 12 + (max_date.month - end_date_iter.month)
            n_month_deposit = min([12, n_month_for_end]) # Сколько месяцев вклада произошло.
            rate_in_date = df_rates.loc[df_rates.date == end_date_iter, "rate"].mean()
            new_income = (rate_in_date * total_money) * (n_month_deposit / 12)
            
            total_money += new_income
            end_date_iter = end_date_iter + pd.DateOffset(months=12)
            income_all_reinvest.append(new_income)
            income_calculate.append([row["start_date"], row['end_date'], row["principal"], rate_in_date, total_money, (n_month_deposit / 12)])
        else:
            break    
    
    income_new_all.append(income_all_reinvest)
    # break

In [ ]:
income_new_all = []
income_calculate = []
for i, row in df_income_correct.iterrows():
    #row["end_date"]
    income_all_reinvest = []
    end_date_iter = row['end_date']
    total_money = row["principal"] + row["income"]
    while True:
        if end_date_iter < max_date:
            n_month_for_end = (max_date.year - end_date_iter.year) * 12 + (max_date.month - end_date_iter.month)
            n_month_deposit = min([12, n_month_for_end]) # Сколько месяцев вклада произошло.
            rate_in_date = df_rates.loc[df_rates.date == end_date_iter, "rate"].mean()
            new_income = (rate_in_date * total_money) * (n_month_deposit / 12)
            
            total_money += new_income
            end_date_iter = end_date_iter + pd.DateOffset(months=12)
            income_all_reinvest.append(new_income)
            income_calculate.append([row["start_date"], row['end_date'], row["principal"], rate_in_date, total_money, (n_month_deposit / 12)])
        else:
            break    
    
    income_new_all.append(income_all_reinvest)
    # break

# Сумма за каждые месяцы

In [105]:
income_new_all_month = []
income_calculate_month = []
for i, row in df_income_correct.iterrows():
    #row["end_date"]
    income_all_reinvest = []
    end_date_iter = row['start_date']
    total_money = row["principal"] # + row["income"]
    while True:
        if end_date_iter < max_date:
            n_month_for_end = (max_date.year - end_date_iter.year) * 12 + (max_date.month - end_date_iter.month)
            n_month_deposit = min([12, n_month_for_end]) # Сколько месяцев вклада произошло.
            rate_in_date = df_rates.loc[df_rates.date == end_date_iter, "rate"].mean()
            new_income = (rate_in_date * total_money) * (n_month_deposit / 12)
            new_income_month = new_income / n_month_deposit
            
            for id_month in range(1, n_month_deposit+1):
                end_date_iter_month = end_date_iter + pd.DateOffset(months=id_month)
                
                
                income_calculate_month.append([row["start_date"], row['end_date'], end_date_iter, end_date_iter_month, row["principal"], rate_in_date, total_money, (n_month_deposit / 12), new_income, total_money, new_income_month])
            
            total_money += new_income
            end_date_iter = end_date_iter + pd.DateOffset(months=12)
            income_all_reinvest.append(new_income)

        else:
            break    
    
    income_new_all_month.append(income_all_reinvest)
    # break

In [106]:
columns = ["start_date", "end_date", "end_date_iter", "end_date_iter_month", "principal", "rate_in_date", "total_money", "cnt_month_dep", "new_income", "total_money", "new_income_month"]

df_income_month = pd.DataFrame(income_calculate_month, columns=columns)

In [107]:
df_income_month.to_excel('df_income_month.xlsx')

In [96]:
df_income_month.groupby("end_date_iter_month").agg({"new_income":"sum"})

,new_income_month
end_date_iter_month,
2013-02-01,8.806751e+02
2013-03-01,1.864848e+03
2013-04-01,2.961202e+03
2013-05-01,4.570989e+03
2013-06-01,6.296277e+03
...,...
2024-08-01,1.368645e+09
2024-09-01,1.574132e+09
2024-10-01,1.788854e+09


In [71]:
df_income_new_all = pd.DataFrame(income_new_all)

In [72]:
df_income_calculate = pd.DataFrame(income_calculate)
df_income_calculate

,0,1,2,3,4,5
0,2012-01-01,2013-01-01,100000,0.09650,120082.101000,1.000000
1,2012-01-01,2013-01-01,100000,0.08310,130060.923593,1.000000
2,2012-01-01,2013-01-01,100000,0.15325,149992.760134,1.000000
3,2012-01-01,2013-01-01,100000,0.09999,164990.536220,1.000000
4,2012-01-01,2013-01-01,100000,0.08400,178849.741262,1.000000
...,...,...,...,...,...,...
307,2022-10-01,2023-10-01,45000,0.19780,54631.083349,0.166667
308,2023-01-01,2024-01-01,45000,0.14793,55256.714246,0.916667
309,2023-04-01,2024-04-01,45000,0.14832,53245.351296,0.666667
310,2023-07-01,2024-07-01,45000,0.16574,51873.008809,0.416667


In [57]:
i = 2
for col in list(df_income_new_all):
     col_name = "income_" + str(i)
     i += 1
     df_income_correct[col_name] = df_income_new_all[col]

In [58]:
df_income_correct[['principal', 'income', 'income_2', 'income_3']].sum().sum()

3095071.4158248147

In [59]:
df_income_correct

,start_date,end_date,principal,rate,income,income_2,income_3,income_4,income_5,income_6,income_7,income_8,income_9,income_10,income_11,income_12,income_13
0,2012-01-01,2013-01-01,100000,0.09514,9514.0000,10568.101000,9978.822593,19931.836541,14997.776086,13859.205042,12957.663754,14441.179524,12224.353606,9800.696005,17663.813811,19994.714519,36061.066063
1,2012-04-01,2013-04-01,45000,0.09538,4292.1000,4899.634740,4668.617948,7754.851467,6427.701049,5695.885748,5039.282621,6312.677844,4889.225079,4401.372111,16477.427599,8893.319661,12335.487239
2,2012-07-01,2013-07-01,45000,0.09936,4471.2000,4581.033120,4867.403592,6505.906286,5973.352076,5257.100644,4901.384366,5819.119069,4044.658146,4845.321337,7359.572292,8110.811006,7716.361457
3,2012-10-01,2013-10-01,45000,0.09400,4230.0000,4265.779500,5083.168968,6064.092745,5598.087369,5083.350476,5120.558086,5211.229506,3705.490095,5748.641811,6342.912480,10327.947064,3685.055475
4,2013-01-01,2014-01-01,45000,0.09650,4342.5000,4100.361750,8190.118563,6162.681702,5694.835609,5324.386553,5933.972630,5023.064743,4027.168401,7258.173580,8215.955525,14817.721687,NaN
5,2013-04-01,2014-04-01,45000,0.09940,4473.0000,4262.098950,7079.599287,5868.010233,5199.917606,4600.488069,5763.002651,4463.496758,4018.123492,15042.658802,8118.935585,11261.377092,NaN
6,2013-07-01,2014-07-01,45000,0.09260,4167.0000,4427.488350,5917.903404,5433.481367,4781.964638,4458.397946,5293.187918,3679.102519,4407.401886,6694.415198,7377.757064,7018.957809,NaN
7,2013-10-01,2014-10-01,45000,0.08665,3899.2500,4646.406735,5543.046385,5117.081690,4646.572647,4680.583260,4763.463900,3387.102464,5254.700010,5797.909031,9440.536621,3368.423652,NaN
8,2014-01-01,2015-01-01,45000,0.08310,3739.5000,7469.328375,5620.320749,5193.648526,4855.801690,5411.739744,4580.998398,3672.748199,6619.401350,7492.891496,13513.654069,NaN,NaN
9,2014-04-01,2015-04-01,45000,0.08615,3876.7500,6439.511812,5337.466102,4729.777703,4184.544360,5241.952566,4059.938838,3654.833084,13682.607606,7384.878648,10243.202740,NaN,NaN


In [60]:
df_income_correct.to_excel("Итоги прибылей.xlsx")

In [61]:
df_income[["principal", "income"]].sum().sum()

2734274.0

In [62]:
df_investments['amount'].sum()

2500000

In [63]:
# Ошибки
# Сумма вкладов. 
# Исправить дату окончания. 